In [4]:
import os
import cv2
import shutil
import argparse
from tqdm import tqdm
import numpy as np
import time
import webbrowser
import threading
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
from pathlib import Path

# --- CRITICAL FIX: Disable OpenCL to prevent GPU crashes ---
cv2.ocl.setUseOpenCL(False)
print("🔧 OpenCL (GPU acceleration) is disabled for stability. Using CPU for processing.")


# --- The Core Stitching Engine (Unchanged) ---
class StitchingEngine:
    """
    A high-fidelity engine for creating detailed panoramic images, optimized for CPU stability.
    """
    def __init__(self, mode='high-quality', stabilize=False, resolution_scale=1.0):
        self.mode = mode
        self.stabilize = stabilize
        self.resolution_scale = resolution_scale
        self.temp_dir = "temp_frames_hq"
        self.max_frames = 20
        self._set_mode_parameters()

    def _set_mode_parameters(self):
        if self.mode == 'ultra-fast':
            self.frame_interval = 150
            self.resolution_scale = min(self.resolution_scale, 0.4)
        elif self.mode == 'fast':
            self.frame_interval = 90
            self.resolution_scale = min(self.resolution_scale, 0.6)
        elif self.mode == 'standard':
            self.frame_interval = 45
            self.resolution_scale = min(self.resolution_scale, 0.8)
        else:
            self.frame_interval = 20
            self.resolution_scale = self.resolution_scale

    def _stabilize_video(self, video_path):
        print("\n⚠️ WARNING: Video stabilization is the most time-consuming step and can take several minutes.")
        print("If your camera was on a tripod, consider running without the --stabilize flag.")
        stabilized_path = os.path.join(self.temp_dir, "stabilized_video.mp4")
        if not os.path.exists(self.temp_dir):
            os.makedirs(self.temp_dir)
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width, height = int(cap.get(3)), int(cap.get(4))
        if self.mode == 'high-quality':
            new_width, new_height = width, height
        else:
            new_width, new_height = int(width * self.resolution_scale), int(height * self.resolution_scale)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(stabilized_path, fourcc, fps, (new_width, new_height))
        ret, prev_frame = cap.read()
        if not ret:
            print("Error: Could not read the first frame.")
            cap.release()
            out.release()
            return video_path
        prev_frame = cv2.resize(prev_frame, (new_width, new_height))
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        transforms = [np.eye(2, 3, dtype=np.float32)]
        nfeatures = 1000 if self.mode == 'high-quality' else 500
        orb = cv2.ORB_create(nfeatures=nfeatures)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("  - Analyzing motion...")
        with tqdm(total=total_frames // 3, unit="steps") as pbar:
            while True:
                for _ in range(3): cap.grab()
                ret, frame = cap.retrieve()
                if not ret: break
                frame = cv2.resize(frame, (new_width, new_height))
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                kp1, des1 = orb.detectAndCompute(prev_gray, None)
                kp2, des2 = orb.detectAndCompute(gray, None)
                if des1 is not None and des2 is not None and len(des1) > 10 and len(des2) > 10:
                    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                    matches = bf.match(des1, des2)
                    if len(matches) > 10:
                        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches[:10]]).reshape(-1, 1, 2)
                        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches[:10]]).reshape(-1, 1, 2)
                        transform, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts)
                        if transform is not None: transforms.append(transform)
                        else: transforms.append(transforms[-1])
                    else: transforms.append(transforms[-1])
                else: transforms.append(transforms[-1])
                prev_gray = gray
                pbar.update(1)
        print("  - Applying smoothing and writing video...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (new_width, new_height))
            out.write(frame)
        smoothing_window = 20
        for i in range(1, len(transforms)):
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(frame, (new_width, new_height))
            start, end = max(0, i - smoothing_window), min(len(transforms), i + smoothing_window)
            smoothed_transform = np.mean(transforms[start:end], axis=0)
            corrected_frame = cv2.warpAffine(frame, smoothed_transform, (new_width, new_height))
            out.write(corrected_frame)
        cap.release()
        out.release()
        print("-> Stabilization complete.")
        return stabilized_path

    def _extract_frames(self, video_path):
        if not os.path.exists(self.temp_dir): os.makedirs(self.temp_dir)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened(): raise FileNotFoundError(f"Could not open video file: {video_path}")
        frame_paths, frame_count, total_frames = [], 0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        dynamic_interval = max(self.frame_interval, total_frames // self.max_frames)
        print(f"Extracting up to {self.max_frames} frames (every {dynamic_interval}th frame) at {self.resolution_scale*100:.0f}% resolution...")
        with tqdm(total=total_frames, unit="frames", disable=None) as pbar:
            while True:
                ret, frame = cap.read()
                if not ret: break
                if frame_count % dynamic_interval == 0:
                    height, width = frame.shape[:2]
                    new_width, new_height = int(width * self.resolution_scale), int(height * self.resolution_scale)
                    resized_frame = cv2.resize(frame, (new_width, new_height))
                    frame_filename = os.path.join(self.temp_dir, f"frame_{frame_count:04d}.jpg")
                    cv2.imwrite(frame_filename, resized_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
                    frame_paths.append(frame_filename)
                    if len(frame_paths) >= self.max_frames:
                        print(f"\nReached maximum frame limit ({self.max_frames}).")
                        break
                frame_count += 1
                pbar.update(1)
        cap.release()
        print(f"-> Extracted {len(frame_paths)} frames.")
        if len(frame_paths) < 2: raise ValueError("Not enough frames were extracted. The video might be too short.")
        return sorted(frame_paths)

    def _stitch_frames(self, frame_paths, output_path):
        if not frame_paths: raise ValueError("No frames provided to stitch.")
        print("\nStarting HIGH-QUALITY stitching process on CPU...")
        print("⏳ This may take several minutes for detailed images. Please be patient.")
        try:
            stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)
            images = [cv2.imread(path) for path in frame_paths]
            status, panorama = stitcher.stitch(images)
            if status != cv2.Stitcher_OK:
                error_map = {
                    cv2.Stitcher_ERR_NEED_MORE_IMGS: "Need more images. Try a smaller frame interval or higher --max-frames.",
                    cv2.Stitcher_ERR_HOMOGRAPHY_EST_FAIL: "Images could not be aligned. Try video stabilization or a different video.",
                    cv2.Stitcher_ERR_CAMERA_PARAMS_ADJUST_FAIL: "Could not adjust camera parameters."
                }
                raise RuntimeError(f"Stitching failed: {error_map.get(status, 'Unknown error')}")
            cv2.imwrite(output_path, panorama, [int(cv2.IMWRITE_JPEG_QUALITY), 98])
            print(f"-> Stitching successful! High-resolution 360° Photo saved to '{output_path}'")
            return True
        except Exception as e: return self._handle_stitching_error(e)

    def _handle_stitching_error(self, error):
        print(f"\n❌ Error: {error}\n--- Suggested Solutions ---")
        print("1. Ensure the video is a smooth 360° pan with consistent speed.")
        print("2. Try running with the --stabilize flag if it was a handheld shot.")
        print("3. Try a smaller frame interval: --interval 15")
        print("4. The video might not have enough unique features for the stitcher to work.")
        print("---------------------------\n")
        return False

    def _safe_delete(self, path, max_retries=3, delay=1):
        for i in range(max_retries):
            try:
                if os.path.isfile(path): os.unlink(path)
                elif os.path.isdir(path): shutil.rmtree(path)
                return True
            except PermissionError:
                if i == max_retries - 1:
                    print(f"\n⚠️ Warning: Could not delete '{path}'. Please delete it manually.")
                    return False
                time.sleep(delay)
        return False

    def create_panorama(self, video_path, output_path):
        source_video = None
        try:
            if self.stabilize: source_video = self._stabilize_video(video_path)
            else: source_video = video_path
            frame_files = self._extract_frames(source_video)
            success = self._stitch_frames(frame_files, output_path)
            return success
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}")
            return False
        finally:
            if os.path.exists(self.temp_dir):
                print("\nCleaning up temporary files...")
                self._safe_delete(self.temp_dir)


# --- AUTOMATED WEB VIEWER ---
class AutomatedWebViewer:
    """
    Creates an HTML file and automatically launches a web server and browser.
    """
    HTML_TEMPLATE = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>360° Photo Viewer</title>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <style>
            body {{ margin: 0; overflow: hidden; font-family: sans-serif; }}
            a-scene {{ display: block; height: 100vh; width: 100vw; }}
        </style>
        <script src="https://aframe.io/releases/1.4.0/aframe.min.js"></script>
    </head>
    <body>
        <a-scene embedded>
            <a-sky src="{image_filename}"></a-sky>
        </a-scene>
    </body>
    </html>
    """
    def __init__(self, image_path):
        self.image_path = Path(image_path)
        self.html_file = self.image_path.with_suffix('.html')
        self.server_port = 8000
        self.server_url = f"http://localhost:{self.server_port}"

    def create_and_view(self):
        """Creates HTML, starts server, and opens browser."""
        if not self.image_path.exists() or self.image_path.stat().st_size == 0:
            print(f"❌ Error: Panorama image not found or is empty at '{self.image_path}'")
            return False

        print(f"\n🌐 Creating HTML viewer: '{self.html_file}'")
        with open(self.html_file, "w") as f:
            f.write(self.HTML_TEMPLATE.format(image_filename=self.image_path.name))
        
        print("✅ HTML file created. Launching viewer...")
        print("-> A local web server is starting and your browser will open automatically.")
        print("-> To stop the server, close this terminal or press CTRL+C in this window.")
        
        # Start server in a separate thread
        server_thread = threading.Thread(target=self._start_server, daemon=True)
        server_thread.start()
        
        # Give server a moment to start, then open browser
        time.sleep(1.5)
        webbrowser.open(self.server_url)
        
        # Keep the main thread alive to serve requests
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("\nServer stopped.")

    def _start_server(self):
        """Starts the simple HTTP server."""
        os.chdir(self.image_path.parent)
        with TCPServer(("", self.server_port), SimpleHTTPRequestHandler) as httpd:
            print(f"Server running at {self.server_url}")
            httpd.serve_forever()


# --- Video Exporter (Unchanged) ---
class PanoramaVideoExporter:
    def __init__(self, panorama_path, output_path="panorama_video.mp4", duration=10, fps=30):
        self.panorama_path = Path(panorama_path)
        self.output_path = Path(output_path)
        self.duration = duration
        self.fps = fps
    def export(self):
        print(f"\n🎬 Creating rotating video from panorama...")
        panorama = cv2.imread(str(self.panorama_path))
        if panorama is None: print(f"❌ Error: Could not read panorama image."); return False
        p_height, p_width, _ = panorama.shape
        output_width = p_width // 2
        output_height = int(output_width * 9 / 16)
        if output_height > p_height: output_height = p_height
        total_frames = self.duration * self.fps
        step = p_width / total_frames
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(str(self.output_path), fourcc, self.fps, (output_width, output_height))
        print(f"  - Generating {total_frames} frames for a {self.duration}s video at {self.fps} FPS...")
        with tqdm(total=total_frames, unit="frames") as pbar:
            for i in range(total_frames):
                offset = int(i * step) % p_width
                if offset + output_width <= p_width: frame = panorama[:, offset : offset + output_width]
                else:
                    part1 = panorama[:, offset:]
                    part2_width = (offset + output_width) - p_width
                    part2 = panorama[:, :part2_width]
                    frame = np.hstack((part1, part2))
                frame = cv2.resize(frame, (output_width, output_height))
                out.write(frame)
                pbar.update(1)
        out.release()
        print(f"✅ Rotating video saved to '{self.output_path}'")
        return True

# --- Manual Input Function (Simplified) ---
def get_manual_input():
    print("\n--- High-Fidelity Panoramic 360° Platform ---")
    input_path = input("Enter path to input video file: ").strip('"')
    while not os.path.exists(input_path):
        print(f"Error: File '{input_path}' not found.")
        input_path = input("Enter path to input video file: ").strip('"')
    output_path = input("Enter output file name (default: 360_photo.jpg): ").strip('"')
    if not output_path: output_path = "360_photo.jpg"
    
    print("\nAvailable quality modes (Default is High-Quality):")
    print("1. ultra-fast - Fastest, lowest quality")
    print("2. fast - Fast, good quality")
    print("3. standard - Balanced")
    print("4. high-quality - Best quality, slowest")
    mode_choice = input("Select mode (1-4, default: 4): ")
    mode_map = {"1": "ultra-fast", "2": "fast", "3": "standard", "4": "high-quality"}
    mode = mode_map.get(mode_choice, "high-quality")

    res_input = input("Output resolution scale (e.g., 1.0 for full, 0.5 for half, default: 1.0): ")
    try:
        resolution = float(res_input)
    except ValueError:
        resolution = 1.0

    stabilize = input("Enable video stabilization? (Very slow, y/n, default: n): ").lower() == 'y'
    interval_input = input("Custom frame interval (leave empty for default): ")
    interval = int(interval_input) if interval_input.isdigit() else None
    
    open_viewer = input("Automatically open the panorama in your browser after creation? (y/n, default: y): ").lower() != 'n'
    create_video = input("Create a rotating video file? (y/n, default: n): ").lower() == 'y'

    return {'input': input_path, 'output': output_path, 'mode': mode, 'resolution': resolution, 'stabilize': stabilize, 'interval': interval, 'open_viewer': open_viewer, 'create_video': create_video}

# --- Command-Line Interface (Simplified) ---
def main():
    parser = argparse.ArgumentParser(description="Create a high-fidelity, interactive 360° photo from a rotating video.", formatter_class=argparse.RawTextHelpFormatter)
    parser.add_argument("--input", "-i", help="Path to the input video file.")
    parser.add_argument("--output", "-o", default="360_photo.jpg", help="Name of the output 360° photo.")
    parser.add_argument("--mode", "-m", choices=['ultra-fast', 'fast', 'standard', 'high-quality'], default='high-quality', help="Quality/Speed mode. Default is high-quality.")
    parser.add_argument("--resolution", "-r", type=float, default=1.0, help="Output resolution scale (e.g., 1.0 for full, 0.5 for half). Default is 1.0.")
    parser.add_argument("--stabilize", "-s", action="store_true", help="Pre-process video to remove camera shake. (WARNING: Very slow)")
    parser.add_argument("--interval", type=int, default=None, help="Override the default frame interval (higher is faster).")
    parser.add_argument("--max-frames", type=int, default=20, help="Maximum number of frames to stitch. Default is 20.")
    parser.add_argument("--no-viewer", action="store_false", dest="viewer", help="Do NOT open the browser viewer.")
    parser.add_argument("--create-video", action="store_true", help="Create a rotating MP4 video from the panorama.")
    parser.add_argument("--video-duration", type=int, default=10, help="Duration of the output video in seconds.")
    parser.add_argument("--manual", action="store_true", help="Use manual input instead of command-line arguments.")

    args = parser.parse_args()

    if args.manual or not args.input:
        params = get_manual_input()
        input_path, output_path, mode, resolution, stabilize, interval, open_viewer, create_video = params['input'], params['output'], params['mode'], params['resolution'], params['stabilize'], params['interval'], params['open_viewer'], params['create_video']
        video_duration = 10
    else:
        input_path, output_path, mode, resolution, stabilize, interval, open_viewer, create_video, video_duration = args.input, args.output, args.mode, args.resolution, args.stabilize, args.interval, args.viewer, args.create_video, args.video_duration

    engine = StitchingEngine(mode=mode, stabilize=stabilize, resolution_scale=resolution)
    if interval: engine.frame_interval = interval
    engine.max_frames = args.max_frames

    print(f"\nMode: {mode} | Resolution: {resolution*100:.0f}% | Stabilization: {'Enabled' if stabilize else 'Disabled'} | Frame Interval: {engine.frame_interval} | Max Frames: {engine.max_frames}")
    print("------------------------------------")
    print("⚠️ NOTICE: This script is now optimized for HIGH QUALITY and CPU STABILITY. Processing will be slower but will not crash.")

    success = engine.create_panorama(input_path, output_path)
    output_file_exists = os.path.exists(output_path)

    if success and output_file_exists:
        print(f"\n✅ Success! Your high-fidelity 360° photo is '{output_path}'")
        
        if open_viewer:
            viewer = AutomatedWebViewer(output_path)
            viewer.create_and_view()
            # The script will now wait here until the server is stopped

        if create_video:
            video_name = r"C:\\Users\\z-pc\\Desktop\\VideoTo3D\\video_data\\VID_20251016_103054.mp4"
            exporter = PanoramaVideoExporter(output_path, output_path=video_name, duration=video_duration)
            exporter.export()
            
    else:
        print("\n❌ Platform could not create the panorama. Please check the error messages above.")

if __name__ == '__main__':
    main()

🔧 OpenCL (GPU acceleration) is disabled for stability. Using CPU for processing.


usage: ipykernel_launcher.py [-h] [--input INPUT] [--output OUTPUT]
                             [--mode {ultra-fast,fast,standard,high-quality}]
                             [--resolution RESOLUTION] [--stabilize]
                             [--interval INTERVAL] [--max-frames MAX_FRAMES]
                             [--no-viewer] [--create-video]
                             [--video-duration VIDEO_DURATION] [--manual]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\z-pc\AppData\Roaming\jupyter\runtime\kernel-v3f93c007085cf40510d0c1f3c7eedbb89f5e28995.json


SystemExit: 2